# Exercise 4: Multi-Agent Handoff — Solution

Complete implementation of the multi-agent handoff workflow with Temporal orchestration.

## Prerequisites

Confirm the Temporal dev server is running and `.env` is configured.

In [1]:
%pip install --quiet temporalio openai rich

import json
import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.common import RetryPolicy
from temporalio.worker import Worker

console = Console()


@activity.defn
def get_weather(location: str) -> str:
    """Return mock weather details for the requested location."""
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    weather = weather_data.get(location, "partly cloudy, 68°F")
    return f"The weather in {location} is {weather}"


@activity.defn
def get_time(location: str) -> str:
    """Return mock local time for the requested location."""
    time_data = {
        "San Francisco": "10:45 AM PST",
        "New York": "1:45 PM EST",
        "London": "6:45 PM GMT",
        "Tokyo": "3:45 AM JST",
    }
    current_time = time_data.get(location, "12:00 PM")
    return f"The current time in {location} is {current_time}"


@activity.defn
async def triage_query(query: str) -> str:
    """Classify the incoming query to the appropriate specialist agent."""
    activity.logger.info("🔍 Triaging query: %s", query)

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a query classifier. Classify queries into one of: "
                    "'weather_agent', 'time_agent', or 'general_agent'. "
                    "Respond with ONLY the agent name, nothing else."
                ),
            },
            {"role": "user", "content": query},
        ],
    )

    agent_name = response.choices[0].message.content.strip()
    activity.logger.info("   → Routed to: %s", agent_name)
    return agent_name


def _safe_parse_arguments(arg_string: str) -> dict:
    try:
        return json.loads(arg_string)
    except json.JSONDecodeError:
        activity.logger.warning("Failed to parse tool arguments, raw input: %s", arg_string)
        return {}


@activity.defn
async def weather_agent(query: str, context: dict) -> str:
    """Handle weather-related queries with tool support."""
    activity.logger.info("🌤️  Weather agent handling query")

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    messages = [
        {"role": "system", "content": "You are a weather specialist assistant."},
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            payload = _safe_parse_arguments(tool_call.function.arguments)
            if tool_call.function.name == "get_weather" and payload.get("location"):
                tool_result = get_weather(**payload)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_result,
                    }
                )

        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        final_message = follow_up.choices[0].message.content
    else:
        final_message = response_message.content

    activity.logger.info("✅ Weather agent completed")
    return final_message


@activity.defn
async def time_agent(query: str, context: dict) -> str:
    """Handle time-related queries with tool support."""
    activity.logger.info("🕐 Time agent handling query")

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_time",
                "description": "Get current time for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    messages = [
        {"role": "system", "content": "You are a time specialist assistant."},
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            payload = _safe_parse_arguments(tool_call.function.arguments)
            if tool_call.function.name == "get_time" and payload.get("location"):
                tool_result = get_time(**payload)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_result,
                    }
                )

        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        final_message = follow_up.choices[0].message.content
    else:
        final_message = response_message.content

    activity.logger.info("✅ Time agent completed")
    return final_message


@workflow.defn
class MultiAgentWorkflow:
    """Workflow orchestrating triage and specialist agent hand-offs."""

    @workflow.run
    async def run(self, query: str, trace_id: str) -> str:
        workflow.logger.info("🚀 Multi-agent workflow started")
        workflow.logger.info("   Query: %s", query)
        workflow.logger.info("   Trace ID: %s", trace_id)

        context = {"history": [], "trace_id": trace_id}

        agent_to_use = await workflow.execute_activity(
            triage_query,
            args=[query],
            start_to_close_timeout=timedelta(seconds=10),
            retry_policy=RetryPolicy(
                maximum_attempts=3,
                initial_interval=timedelta(seconds=1),
            ),
        )

        workflow.logger.info("   Routing to: %s", agent_to_use)

        if agent_to_use == "weather_agent":
            result = await workflow.execute_activity(
                weather_agent,
                args=[query, context],
                start_to_close_timeout=timedelta(seconds=30),
                retry_policy=RetryPolicy(maximum_attempts=3),
            )
        elif agent_to_use == "time_agent":
            result = await workflow.execute_activity(
                time_agent,
                args=[query, context],
                start_to_close_timeout=timedelta(seconds=30),
                retry_policy=RetryPolicy(maximum_attempts=3),
            )
        else:
            result = "Query routed to general agent (not implemented in this exercise)"

        workflow.logger.info("✅ Multi-agent workflow completed")
        return result


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Run the Solution

Execute the solution workflow to observe agent handoffs.

In [2]:
# Run the multi-agent handoff solution
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 4: Multi-Agent Handoff — Solution[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "multi-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[MultiAgentWorkflow],
        activities=[triage_query, weather_agent, time_agent, get_weather, get_time],
    ):
        query = "What's the weather like in London and what time is it there?"
        workflow_id = f"multi-agent-{trace_id}"

        console.print(f"[yellow]Query:[/yellow] {query}\n")

        result = await client.execute_workflow(
            MultiAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]✅ Final Response:[/bold green]\n{result}\n")
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    asyncio.run(run_solution())
else:
    import nest_asyncio
    nest_asyncio.apply()
    task = loop.create_task(run_solution())
    await task

🚀 Exercise 4: Multi-Agent Handoff — Solution

Trace ID: ccd9862a-abc0-4085-af52-72256b43a044

ValueError: Activity get_weather is not async so an activity_executor must be present

## Source Code

The workflow orchestration and activity implementations are defined above in the first code cell.